In [39]:
import gc
gc.collect()
from typing import Type
from pydantic import confloat
import numpy as np
import pandas as pd 
import geopandas as gpd
import h3
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process
import uuid
import warnings
warnings.filterwarnings('ignore')
import datetime
import shapely.speedups
import statsmodels.formula.api as smf


In [40]:
## Structuring & adding needed columns
school = pd.read_csv('/Users/shilpa/Documents/UNICEF/data/school/kazakshtan/kazakhstan_gov_042021_rur.csv', header=0)
pop = pd.read_csv("/Users/shilpa/Documents/UNICEF/data/socio-economic/kazakshtan/raw/population_kaz_2018-10-01.csv", header=0)
eco = pd.read_csv("/Users/shilpa/Documents/UNICEF/data/socio-economic/kazakshtan/raw/kaz_relative_wealth_index.csv", header=0)
countries_gdf = gpd.read_file("/Users/shilpa/Documents/UNICEF/data/gadm/gadm36_KAZ_gpkg/gadm36_KAZ.gpkg", layer='gadm36_RWA_0')


In [43]:
print(school['connectivity_speed'].isna().sum())
print(school.head())
print(eco['error'].describe(include='all'))
school['rur'] = np.where((school['fid'].isna()==True), "rural", "urban")
print(school['rur'].value_counts())
school = school.drop(['field_13','fid','ID_HDC_G0', 'NAME_MAIN', 'NAME_LIST', 'POP_2015','BU_2015'], axis = 1)


29
   school_id               admin1              admin2         admin3  \
0          0  Акмолинская область    Степногорск Г.А.  г.Степногорск   
1          1  Акмолинская область   Aршалынский район       п.Аршалы   
2          2  Акмолинская область   Атбасарский район      г.Атбасар   
3          3  Акмолинская область   Жаркаинский район   г.Державинск   
4          4  Акмолинская область  Зерендинский район    с.Гранитный   

                                                name  num_students        lat  \
0  КГУ "Вечерняя (сменная) средняя общеобразовате...          60.0  52.469684   
1  ГУ"Вечерняя (сменная) средняя общеобразователь...          15.0  50.832731   
2  ГУ "Вечерняя (сменная) общеобразовательная сре...          68.0  51.775718   
3  ГУ "Жаркаинская вечерняя (сменная) общеобразов...          26.0  51.098673   
4  ГУ "Вечерняя (сменная) средняя общеобразовател...          79.0  53.434238   

         lon  num_computers  num_computers_education  num_interactive_boards 

In [44]:
print(school.head())
print(pop.head())
print(eco.head())

#Add hex id
list_hex_res = []
for i in range(0, len(school)):
    # index the point in the H3 hexagon of given index resolution
    h = h3.geo_to_h3(lat = school['lat'][i],
                     lng = school['lon'][i],
                     resolution = 7)
    list_hex_res.append(h)

school['hex7'] = list_hex_res

#Add hex id
list_hex_res = []
for i in range(0, len(pop)):
    # index the point in the H3 hexagon of given index resolution
    h = h3.geo_to_h3(lat = pop['latitude'][i],
                     lng = pop['longitude'][i],
                     resolution = 7)
    list_hex_res.append(h)

pop['hex7'] = list_hex_res


#Add hex id
list_hex_res = []
for i in range(0, len(eco)):
    # index the point in the H3 hexagon of given index resolution
    h = h3.geo_to_h3(lat = eco['latitude'][i],
                     lng = eco['longitude'][i],
                     resolution = 7)
    list_hex_res.append(h)

eco['hex7'] = list_hex_res



   school_id               admin1              admin2         admin3  \
0          0  Акмолинская область    Степногорск Г.А.  г.Степногорск   
1          1  Акмолинская область   Aршалынский район       п.Аршалы   
2          2  Акмолинская область   Атбасарский район      г.Атбасар   
3          3  Акмолинская область   Жаркаинский район   г.Державинск   
4          4  Акмолинская область  Зерендинский район    с.Гранитный   

                                                name  num_students        lat  \
0  КГУ "Вечерняя (сменная) средняя общеобразовате...          60.0  52.469684   
1  ГУ"Вечерняя (сменная) средняя общеобразователь...          15.0  50.832731   
2  ГУ "Вечерняя (сменная) общеобразовательная сре...          68.0  51.775718   
3  ГУ "Жаркаинская вечерняя (сменная) общеобразов...          26.0  51.098673   
4  ГУ "Вечерняя (сменная) средняя общеобразовател...          79.0  53.434238   

         lon  num_computers  num_computers_education  num_interactive_boards  \


In [30]:
## Aggregates
school_grid = school.groupby("hex7").agg( connectivity_speed=pd.NamedAgg(column="connectivity_speed", aggfunc="mean"),school_count=pd.NamedAgg(column="school_id", aggfunc="size"),)
pop_grid = pop.groupby("hex7").agg(population_2020=pd.NamedAgg(column="population_2020", aggfunc="sum"),pop_count=pd.NamedAgg(column="population_2020", aggfunc="size"),)
eco_grid = eco.groupby("hex7").agg(rwi=pd.NamedAgg(column="rwi", aggfunc="mean"),eco_count=pd.NamedAgg(column="rwi", aggfunc="size"),error=pd.NamedAgg(column="error", aggfunc="mean"),)


In [36]:
## Merge
df = pd.merge(school_grid,pop_grid,on='hex7')
df1 = pd.merge(df,eco_grid,on='hex7')
df1

,connectivity_speed,school_count,population_2020,pop_count,rwi,eco_count,error
hex7,,,,,,,
8710900d5ffffff,11.000,1,538.975593,354,0.743,1,0.3640
8710900eeffffff,10.500,2,2566.985452,1686,0.276,2,0.3535
8710900f5ffffff,4.000,1,80.694086,53,-0.338,1,0.2470
871090101ffffff,3.000,1,95.919385,63,-0.235,1,0.2630
871090201ffffff,4.000,1,36.540718,24,-0.617,1,0.2070
...,...,...,...,...,...,...,...
872c6e171ffffff,11.000,1,194.256280,80,-0.421,1,0.3100
872c6e261ffffff,11.000,1,214.179820,362,0.080,1,0.4000
872c6e312ffffff,10.000,1,366.658729,151,-0.584,1,0.4000


In [38]:
## Analysis
display(df1.corr())

lm = smf.ols('connectivity_speed~school_count+population_2020+rwi+error', data=df1).fit()
print(lm.summary())

,connectivity_speed,school_count,population_2020,pop_count,rwi,eco_count,error
connectivity_speed,1.000000,0.183616,0.283000,0.310730,0.346792,0.195541,0.334728
school_count,0.183616,1.000000,0.539518,0.548289,0.431470,0.211326,0.258049
population_2020,0.283000,0.539518,1.000000,0.765672,0.585376,0.280059,0.366869
pop_count,0.310730,0.548289,0.765672,1.000000,0.678640,0.378116,0.479995
rwi,0.346792,0.431470,0.585376,0.678640,1.000000,0.272614,0.517569
eco_count,0.195541,0.211326,0.280059,0.378116,0.272614,1.000000,0.211013
error,0.334728,0.258049,0.366869,0.479995,0.517569,0.211013,1.000000


                            OLS Regression Results                            
Dep. Variable:     connectivity_speed   R-squared:                       0.160
Model:                            OLS   Adj. R-squared:                  0.159
Method:                 Least Squares   F-statistic:                     208.0
Date:                Thu, 17 Jun 2021   Prob (F-statistic):          1.57e-163
Time:                        01:56:12   Log-Likelihood:                -10506.
No. Observations:                4376   AIC:                         2.102e+04
Df Residuals:                    4371   BIC:                         2.105e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           3.9621      0.250     